In [5]:
import pickle

X_train = pickle.load(open('../dataset/processed-data/X_train.pkl', 'rb'))
y_train = pickle.load(open('../dataset/processed-data/y_train.pkl', 'rb'))

X_test = pickle.load(open('../dataset/processed-data/X_test.pkl', 'rb'))
y_test = pickle.load(open('../dataset/processed-data/y_test.pkl', 'rb'))

In [6]:
print(len(X_train), len(X_test))

4736 2030


In [7]:
X_train[0]

'loạt hoạt_động trải nghiệm tại tuần_lễ du_lịch bà rịa vũng_tàu tuâ lê du li ch vơ nhiều hoạt_động như lễ_hội diều hội_chợ ẩm_thực trình_diễn nghệ_thuật diễn ra tư nga dự_kiến thu_hút hàng nghìn du_khách với chủ_đề let go ba ria vung tau tuần_lễ du_lịch nhằm thu_hút khách du_lịch trong nước và quốc_tế góp_phần phát_triển kinh_tế xã_hội của tỉnh thời_điểm mùa thu thời_tiết mát_mẻ dễ_chịu nhất trong năm việc tổ_chức sự_kiện làm cho du_khách nhớ đến một bà rịa vũng_tàu hấp_dẫn quanh_năm bà trần thị thu hiền phó_giám_đốc sở du_lịch chia_sẻ tại họp_báo hôm chuỗi hoạt_động của sự_kiện được thiết_kế đa_dạng liên_tục tại khu_vực công_viên san_hô xanh biển đông bãi sau thành_phố vũng_tàu và khu_vực biển_hồ tràm huyện xuyên mộc với nghĩa mỗi ngày là một trải nghiệm mới_lạ cho du_khách trong đó các hoạt_động nổi_bật là lễ_hội diều lướt_ván diều triển_lãm các tiểu_cảnh và mô_hình_tượng cát hội_chợ ẩm_thực và bia hội_chợ ocop one commune one product mỗi xã một sản_phẩm giới_thiệu nhiều đặc_sản của 

In [8]:
y_train[0]

'du-lich'

# - Count Vectors as features

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
# Create a count vectorizer object
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_train)

# Transform the training and testing data using count vectorizer object
X_train_count = count_vect.transform(X_train)
X_test_count = count_vect.transform(X_test)

# - Tf-Idf Vectors as features

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

## Word level - we choose max number of words equal to 30000 except all words (100k+ words)

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)

# Assume that we don't have test set before
X_test_tfidf = tfidf_vect.transform(X_test)

## N-gram level - we choose max number of words equal to 30000 except all words (100k+ words)

In [ ]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_train)
X_train_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)

# Assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

## N-gram-char level - we choose max number of words equal to 30000 except all words (100k+ words)

In [ ]:
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(X_train)
X_train_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_train)

# Assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_test)

# - Singular value decomposition

In [ ]:
from sklearn.decomposition import TruncatedSVD

## Word level

In [ ]:
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_train_tfidf)

X_train_tfidf_svd = svd.transform(X_train_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

## N-gram level

In [ ]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_train_tfidf_ngram)

X_train_tfidf_ngram_svd = svd_ngram.transform(X_train_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)

## N-gram Char level

In [ ]:
svd_ngram_char = TruncatedSVD(n_components=300, random_state=42)
svd_ngram_char.fit(X_train_tfidf_ngram_char)

X_train_tfidf_ngram_char_svd = svd_ngram_char.transform(X_train_tfidf_ngram_char)
X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)

# - Word Embeddings

In [ ]:
from gensim.models import KeyedVectors
import os

In [ ]:
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
word2vec_model_path = os.path.join(dir_path, "dataset/vi/wordvectors/data/vi.bin")

w2v = KeyedVectors.load(word2vec_model_path)
vocab = list(w2v.wv.index_to_key)
wv = w2v.wv

def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
                sentence.append(wv[word])
        word2vec_data.append(sentence)
    return(word2vec_data)

X_train_w2v = get_word2vec_data(X_train)
X_test_w2v = get_word2vec_data(X_test)